# Часть 2. Предсказание количества ретвитов твита в выборке RuTweetCorp


## Загрузим и изучим данные 

In [2]:
import pandas as pd


In [3]:
columns = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'trtw', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount']
pos = pd.read_csv('positive.csv', sep=';', names=columns)
print(pos.shape)
neg = pd.read_csv('negative.csv', sep=';', names=columns)
print(neg.shape)

(114911, 12)
(111923, 12)


In [4]:
data = pos.append(neg)


Посмотрим на твиты с большим количеством ретвитов


In [5]:
data.sort_values(by=['trtw'], ascending=False).head()

,id,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
104932,411125460018683904,1386854922,angelinagunbina,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13817,0,7005,1719,1887,1
80674,410748015344091136,1386764932,Jake_and_Cake,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13793,0,11857,526,254,9
80624,410747573662523392,1386764827,rainydamon,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13791,0,2047,769,680,0
79028,410727313773723648,1386759997,run_fools,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13789,0,751,59,75,0
75569,410648531922616320,1386741214,EshikFromMars,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13762,0,12099,156,93,2


Как видим, наибольшее количество ретвитов твита пользователя @JaredLeto и у всех разное количество ретвитов, то есть мы видим срез количества на определённую дату. Попробуем найти этот первичный твит в выборке

In [6]:
data.loc[(data['ttext'].str.contains('Моя русская семья,')) & (~data['ttext'].str.contains('RT'))].sort_values(by=['trtw', 'tdate'])


,id,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount


К сожалению, его нету в базе и мы не сможем делать прогноз, не зная вообще никаких параметров постящего пользователя. 
Выделим те сообщения, которые есть в базе и у которых есть ретвиты.  

In [7]:
data_shape = data.loc[(data['trtw'] >= 0 ) & (~data['ttext'].str.contains('RT'))].sort_values(by=['trtw', 'tdate'])
data_shape['trtw'].value_counts()

0    189035
1       634
2         2
Name: trtw, dtype: int64

Выборка резко сократилась, теперь у нас несбалансированные классы. Посчитаем сразу, что в сообщениях с 2-мя ретвитами ретвитов на самом деле 1, так как с 2-мя объектами класса мы не сможем сгенерировать в будущем синтетические данные.


In [8]:
data_shape.loc[data_shape['trtw'] == 2, 'trtw'] = 1
data_shape['trtw'].value_counts()



0    189035
1       636
Name: trtw, dtype: int64

Оставим в выборке только те признаки, от значения которых в будущем не зависит настоящее.  

In [9]:
data = data_shape[['tmane', 'ttext', 'ttype', 'tstcount', 'tfol', 'tfrien', 'listcount']]
y = data_shape['trtw']

Добавим новые признаки, количество слов и длина твита

In [10]:
def number_of_words(doc):
    
    return len(doc.split(" "))


In [11]:
data['numwords'] = data['ttext'].apply(number_of_words)
data['lentweet'] = data['ttext'].apply(lambda x: len(x))


C:\Users\Alex\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Alex\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Попробуем что-нибудь с делать с балансировкой классов. 
Я воспользуюсь алгоритмом ADASYN, он более адаптивен к выборкам, чем SMOTE.

In [12]:
from imblearn.over_sampling import ADASYN

ada = ADASYN(random_state=42)


Подготовим pipeline, сшивающий векторизированные текстовые данные(текст и имя, имя тоже будем принимать за параметр) и остальные признаки.

In [13]:
from sklearn.compose import ColumnTransformer

from sklearn.feature_extraction.text import CountVectorizer


preprocessor = ColumnTransformer(
        transformers=[('text', CountVectorizer(), 'ttext'), ('name', CountVectorizer(), 'tmane')])
data_trans = preprocessor.fit_transform(data)


В дальнейшее обучении будем использовать не все признаки, а 50 важнейших, подберём их, используя SVD-метод. 50 я взял как число, после которого мой компьютер не зависает при дальнейших вычислениях)


In [20]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import FeatureUnion

svd = TruncatedSVD(n_components=50, n_iter=4, random_state=42)
X_features = svd.fit_transform(data_trans)


Сгенерим синтетические данные

In [21]:
X_train, y_train = ada.fit_resample(X_features, y)

## Обучение модели

In [17]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline


In [22]:
mae = make_scorer(mean_absolute_error)

pipe = Pipeline([
    ('scale', StandardScaler()),
    ('regr', Lasso())
])

param_grid = [
    {
        'regr': [Lasso(), Ridge()],
        'regr__alpha': np.logspace(-4, 1, 6),
    },
    {
        'regr': [SGDRegressor()],
        'regr__alpha': np.logspace(-5, 0, 6),
        'regr__max_iter': [500, 1000],
    },
    
   
     {   'regr': [XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)],
        'regr__min_child_weight': [0.5, 1],
        'regr__gamma': [5, 10],
        'regr__subsample': [0.7, 1.0],
        'regr__colsample_bytree': [0.5, 1.0],
        'regr__max_depth': [8, 12]
        }
]


grid = GridSearchCV(pipe, param_grid=param_grid, cv=3, n_jobs=-1, verbose=10, scoring=mae)
grid.fit(X_train, y_train)


print("Best score: %0.6f" % grid.best_score_)
print("Best parameters set:")
best_parameters = grid.best_estimator_.get_params()
print(best_parameters)


Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   42.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   48.0s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   59.3s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 30.4min
[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed: 39.9min finished


Best score: 0.666667
Best parameters set:
{'memory': None, 'steps': [('scale', StandardScaler()), ('regr', Lasso(alpha=0.1))], 'verbose': False, 'scale': StandardScaler(), 'regr': Lasso(alpha=0.1), 'scale__copy': True, 'scale__with_mean': True, 'scale__with_std': True, 'regr__alpha': 0.1, 'regr__copy_X': True, 'regr__fit_intercept': True, 'regr__max_iter': 1000, 'regr__normalize': False, 'regr__positive': False, 'regr__precompute': False, 'regr__random_state': None, 'regr__selection': 'cyclic', 'regr__tol': 0.0001, 'regr__warm_start': False}


Итого, лучший результат по MAE -- 0.66

Дальнейшие возможные улучшения:
* Использование временных характеристик, например время постинга твита
* Использование признака частоты постинга твитов пользователем

итд